In [1]:
import math
import torch
import torch.nn as nn
from torch.autograd import Variable

import pandas as pd
import seaborn as sns
import numpy as np

import matplotlib.pyplot as plt
from collections import OrderedDict

sns.set_style("white")
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [2]:
# The reynolds numbers from 10 to 90 are used for training 
# The model is evaluated on the last 10 reynolds numbers from 91 to 100

outputs = np.zeros((81, 1803))
for Re in range(10,91,1):
    outputs[Re-10] = np.loadtxt('pinns/weights/weights_'+str(Re)+'.txt')
    
inputs = np.arange(10,91,1)
inputs = np.reshape(inputs, (inputs.shape[0],1))
inputs = Variable(torch.from_numpy(inputs).float(), requires_grad=False).to(device)
outputs = Variable(torch.from_numpy(outputs).float(), requires_grad=False).to(device)

In [3]:
class HyperNetwork(nn.Module):
    def __init__(self):
        super(HyperNetwork, self).__init__()
        
        self.layers = nn.Sequential(nn.Linear(1,512),
                      nn.ReLU(),             
                      nn.Linear(512,256),
                      nn.ReLU(),
                      nn.Linear(256,256),
                      nn.ReLU(),
                      nn.Linear(256,128),
                      nn.ReLU(),
                      nn.Linear(128,1803))    
        
    def forward(self, Re):        
        weights = self.layers(Re)
        return weights

In [4]:
hnet = HyperNetwork()
hnet = hnet.to(device)
mse_cost_function = nn.MSELoss() 
optimizer = torch.optim.Adam(hnet.parameters())

In [5]:
iterations = 10000
for epoch in range(iterations):
    optimizer.zero_grad()
    
    hnet_out = hnet.forward(inputs)
    Loss = mse_cost_function(hnet_out, outputs)
    Loss.backward()
    optimizer.step()

    with torch.autograd.no_grad():
        if epoch%100 == 0:
            print('Iter %d, Loss: %.4e' % (epoch, Loss))

Iter 0, Loss: 6.3884e-01
Iter 100, Loss: 7.0518e-02
Iter 200, Loss: 6.9959e-02
Iter 300, Loss: 6.9598e-02
Iter 400, Loss: 6.9394e-02
Iter 500, Loss: 6.9291e-02
Iter 600, Loss: 6.9248e-02
Iter 700, Loss: 6.9224e-02
Iter 800, Loss: 6.9208e-02
Iter 900, Loss: 6.9193e-02
Iter 1000, Loss: 6.9181e-02
Iter 1100, Loss: 6.9170e-02
Iter 1200, Loss: 6.9308e-02
Iter 1300, Loss: 6.9151e-02
Iter 1400, Loss: 6.9141e-02
Iter 1500, Loss: 6.9129e-02
Iter 1600, Loss: 6.9169e-02
Iter 1700, Loss: 6.9172e-02
Iter 1800, Loss: 6.9141e-02
Iter 1900, Loss: 6.9138e-02
Iter 2000, Loss: 6.9127e-02
Iter 2100, Loss: 6.9206e-02
Iter 2200, Loss: 6.9121e-02
Iter 2300, Loss: 6.9111e-02
Iter 2400, Loss: 6.9156e-02
Iter 2500, Loss: 6.9108e-02
Iter 2600, Loss: 6.9101e-02
Iter 2700, Loss: 6.9108e-02
Iter 2800, Loss: 6.9087e-02
Iter 2900, Loss: 7.3407e-02
Iter 3000, Loss: 7.0259e-02
Iter 3100, Loss: 7.0220e-02
Iter 3200, Loss: 7.0211e-02
Iter 3300, Loss: 7.0209e-02
Iter 3400, Loss: 7.0209e-02
Iter 3500, Loss: 7.0209e-02
Iter

In [6]:
outputs = np.zeros((10, 1803))
for Re in range(91,101,1):
    outputs[Re-91] = np.loadtxt('pinns/weights/weights_'+str(Re)+'.txt')
    
inputs = np.arange(91,101,1)
inputs = np.reshape(inputs, (inputs.shape[0],1))
inputs = Variable(torch.from_numpy(inputs).float(), requires_grad=False).to(device)
outputs = Variable(torch.from_numpy(outputs).float(), requires_grad=False).to(device)
pred_outputs = hnet.forward(inputs)
L2_error = torch.linalg.norm(outputs-pred_outputs,2)/torch.linalg.norm(outputs,2)
print('Relative L2 Error: %e \n' % (L2_error))

Relative L2 Error: 9.996828e-01 

